In [1]:
from pyarrow import fs, hdfs
import pyarrow.csv as pv
import collections
import re
from datetime import datetime
hdfs.HadoopFileSystem("hdfs://namenode:9000")
connection = fs.HadoopFileSystem(host="namenode", port=9000)

/tmp/ipykernel_3487/23586936.py:2: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  hdfs.HadoopFileSystem("hdfs://namenode:9000")


In [16]:
connection.get_file_info(fs.FileSelector('/', recursive=True))

[<FileInfo for '/pasta_avo1': type=FileType.Directory>,
 <FileInfo for '/pasta_avo1/sentiment140.csv': type=FileType.File, size=44326223>,
 <FileInfo for '/pasta_avo2': type=FileType.Directory>,
 <FileInfo for '/pasta_avo2/pasta_pai': type=FileType.Directory>]

In [4]:
connection.create_dir('/pasta_avo1')

In [15]:
source_file_path = '/home/jovyan/work/sentiment140.csv'
destination_file_path = '/pasta_avo1/sentiment140.csv'

with connection.open_output_stream(destination_file_path) as stream:
    stream.write(open(source_file_path, 'rb').read())

In [17]:
with connection.open_input_stream(destination_file_path) as f:
    print(f.read(500))

b'\xef\xbb\xbftarget;ids;date;flag;user;text\r\n0;1467810369;Mon Apr 06 22:19:45 PDT 2009;NO_QUERY;_TheSpecialOne_;"@switchfoot http://twitpic.com/2y1zl - Awww, that\'s a bummer.  You shoulda got David Carr of Third Day to do it. ;D"\r\n0;1467810672;Mon Apr 06 22:19:49 PDT 2009;NO_QUERY;scotthamilton;is upset that he can\'t update his Facebook by texting it... and might cry as a result  School today also. Blah!\r\n0;1467810917;Mon Apr 06 22:19:53 PDT 2009;NO_QUERY;mattycus;@Kenichan I dived many times for the ball'


In [30]:
table = pv.read_csv(connection.open_input_file(destination_file_path), parse_options=pv.ParseOptions(delimiter=';'))

tweets = table.column(5).to_pylist()

word_counts = collections.Counter()

for tweet in tweets:
    words = re.findall(r'\b\w+\b', tweet.lower())
    word_counts.update(words)

print("Palavras mais frequentes")
for word, count in word_counts.most_common(10):
    print(f'{word}: {count}')

palavras mais frequentes
i: 226118
to: 127466
the: 104841
my: 74810
a: 73919
it: 63174
and: 62151
is: 52001
t: 49829
in: 46649


In [31]:
mention_counts = collections.Counter()

for tweet in tweets:
    mentions = re.findall(r'@\w+', tweet)
    mention_counts.update(mentions)

print("Usuarios mais mencionados:")
for mention, count in mention_counts.most_common(10):
    print(f'{mention}: {count}')

Usuarios mais mencionados:
@mileycyrus: 719
@tommcfly: 664
@ddlovato: 417
@JonathanRKnight: 221
@taylorswift13: 200
@mitchelmusso: 192
@DavidArchie: 185
@jordanknight: 159
@selenagomez: 144
@DonnieWahlberg: 127


In [29]:
hashtag_counts = collections.Counter()

for tweet in tweets:
    hashtags = re.findall(r'#\w+', tweet)
    hashtag_counts.update(hashtags)

print("Hashtags mais frequentes:")
for hashtag, count in hashtag_counts.most_common(10):
    print(f'{hashtag}: {count}')


Hashtags mais frequentes:
#fb: 373
#fail: 115
#asot400: 113
#bgt: 73
#1: 70
#2: 64
#followfriday: 61
#NOME: 55
#myweakness: 54
#e3: 52


In [ ]:
dates = table.column(3).to_pylist()
days_of_week = [datetime.strptime(date, '%a %b %d %H:%M:%S PDT %Y').strftime('%A') for date in dates]
words_by_day = {day: collections.Counter() for day in set(days_of_week)}

for tweet, day in zip(tweets, days_of_week):
    words = re.findall(r'\b\w+\b', tweet.lower())
    words_by_day[day].update(words)

for day, counter in words_by_day.items():
    print(f"Palavras mais frequentes em {day}:")
    for word, count in counter.most_common(10):
        print(f'{word}: {count}')

In [ ]:
hours_of_day = [datetime.strptime(date, '%a %b %d %H:%M:%S PDT %Y').strftime('%H') for date in dates]
hashtags_by_hour = {hour: collections.Counter() for hour in set(hours_of_day)}

for tweet, hour in zip(tweets, hours_of_day):
    hashtags = re.findall(r'#\w+', tweet)
    hashtags_by_hour[hour].update(hashtags)

for hour, counter in hashtags_by_hour.items():
    print(f"Hashtags mais frequentes na hora {hour}:00:")
    for hashtag, count in counter.most_common(10):
        print(f'{hashtag}: {count}')